### 手動でやること

In [1]:
!pip install --upgrade --quiet google-cloud-aiplatform google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.4/206.4 kB 7.2 MB/s eta 0:00:00


In [2]:
import os
import glob
import numpy as np
import pandas as pd

import seaborn as sns
!pip install japanize-matplotlib
import matplotlib.pyplot as plt
import japanize_matplotlib

from google.colab import drive
drive.mount('/content/drive')
from natsort import natsorted

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=59e200c3337f9e364711e82ba0020b44119611c034f380a4b3e44326440a192d
  Stored in directory: /root/.cache/pip/wheels/da/a1/71/b8faeb93276fed10edffcca20746f1ef6f8d9e071eee8425fc
Successfully built japanize-matplotlib
Mounted at /content/drive


1. 下記リンクの共有フォルダのショートカットを「マイドライブ」に作成
https://drive.google.com/drive/folders/1PIFRHDW4Ra_gWnqkxj7uA5gcQLUxZWWq
<br>※「マイドライブ」に作成しない場合、次セルでエラーが発生する

In [3]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

In [4]:
PROJECT_ID = "lecture-1-443205"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

In [5]:
# フォルダのパスを設定
# 個人フォルダ指定のため、分析者によって個別指定要
file_path_list = glob.glob('/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第*回_*確定*xlsx')
file_path_list = natsorted(file_path_list)
file_path_list

['/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第0回_回答結果0430_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第1回_講師塚本_回答結果0513_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第2回_講師志潟_回答結果0513_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第3回_講師熊澤_回答結果0521_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第4回_講師嵐_回答結果0529_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第5回_講師中内_回答結果0603_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/00.（内部）運営資料/103.アンケート データ分析/02.2025Summer rawデータ_個人情報抜き/GCI2025Summer_第6回_講師熊澤_回答結果0610_確定.xlsx',
 '/content/drive/MyDrive/GCI2025_Summer/

2. 次セルに分析する講義回の数字を入力

In [6]:
section_num = 8 # @param {type:"integer"}

### ここから自動

In [7]:
df = pd.read_excel(file_path_list[section_num])

In [8]:
# コメントを抽出する
feedback_columns1 = df['【必須】本日の講義で学んだことを50文字以上で入力してください。']
feedback_columns2 = df['（任意）本日の講義で特によかった部分について、具体的にお教えください。'].dropna()
feedback_columns3 = df['（任意）分かりにくかった部分や改善点などがあれば、具体的にお教えください。'].dropna()
feedback_columns4 = df['（任意）講師について、よかった点や不満があった点などについて、具体的にお教えください。'].dropna()
feedback_columns5 = df['（任意）今後開講してほしい講義・分野などがあればお書きください。'].dropna()
feedback_columns6 = df['（任意）ご自由にご意見をお書きください。'].dropna()

In [9]:
def clean(comment):
    # フィードバックデータをフィルタリング
    filtered_feedback = comment[
        ~comment.str.contains(r'^(無し|なし|ありません)$') &
        (comment.str.len() > 15)
    ]

    # インデックスをリセット
    filtered_feedback.reset_index(drop=True, inplace=True)

    # 1行目にもハイフンをつけるため、'- 'を先頭に追加
    output = '-' + '\n-'.join(filtered_feedback).replace(' ', '').replace('　', '')
    return output

In [10]:
feedback_content1 = clean(feedback_columns2)
feedback_content2 = clean(feedback_columns3)
feedback_content = feedback_content1 + '\n' + feedback_content2

/tmp/ipython-input-9-2091982815.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~comment.str.contains(r'^(無し|なし|ありません)$') &
/tmp/ipython-input-9-2091982815.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~comment.str.contains(r'^(無し|なし|ありません)$') &


In [11]:
print(feedback_content)

-講師を2段構えで用意しており、それぞれの実体験を元にした話しで解像度が深まった点。
-この講義の内容は大学の授業や実験で扱っている内容だったので大変ためになった。今までよくわかっていなかったもの(特にk-means法や主成分分析など)の理解が進み、わかりやすかった。
-教師なし学習と教師あり学習の違いがよくわかりました。
-難しい内容だったので何度も繰り返したい
-講義の最後に，GCI修了後の流れや機械学習の応用例の紹介があり，学習のモチベーションにつながった．
-inputの量が適切だったと思います。
-講師の方々がわかりやすく教えてくださったこと、また、時間配分を事前にご教示いただけて計画的に学習に臨めました。
-内容は非常に難しかったが、教師なし学習がどのようなものか初歩的なイメージをつけることができた。
-講義の冒頭で今回の学習目標を明確に示していただけたため、内容を整理しながら効率的に理解することができました。また、概念的な説明の後に、具体的なコード実装による解説が行われたことで、理論と実践のつながりが強まり、理解がより深まりました。
-クラスタリングや主成分分析による次元削減の話はコンペにも使えそうだなと感じたので、試してみようと思った。
-時系列データなどについて学ぶことでより汎用的な知識を得たと感じた。
-研究でクラスタリングは用いていたが主成分分析についてはずっと曖昧にしていた部分であったため今回の講義で学べて大変有意義であった。
-目的や使い方の概要を理解できた。
-難しいところ(k-means法)では図を多く用いて説明されていたので，わかりやすかったところがよかったです．\nポイントの部分を詳しくゆっくり解説してくれていたので，わかりやすくて良かったです．
-実際のデータを使ってデータを操作してみることができたのが良かったです。
-毎回ともスライドが非常に整理されており学習の助けになっている。
-内容は高度だったが、時系列データの取扱いについての話があったこと。また、時系列モデリングについての整理がなされていること。
-前半の話は、とても分かりやすかったです。
-時系列データについて知れたこと。
-各々、系統的に整理できてよかったです。
-STL分解やADF検定などの具体的な手法の紹介があり、時系列データの前処理や確認手順が整理できた点が良か

In [12]:
feedback_lecturer = clean(feedback_columns4)

/tmp/ipython-input-9-2091982815.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ~comment.str.contains(r'^(無し|なし|ありません)$') &


In [13]:
print(feedback_lecturer)

-特に後半の講師の方はTAのリーダーをされていることもあり明瞭な話し方で分かりやすかった。
-何が大切なのかがよくわからなかったです。
-課題について軽く説明する義務があると思う．
-教師なし学習について、これまで学んできた教師あり学習と比較しながら解説していただいたことで、両者の違いが明確になり、理解がスムーズに進みました。また、各手法の実務での活用例を交えて説明していただけたことで、より現実的なイメージを持つことができました。
-前半、後半の方とも「ま」「まあ」などのフィラーが多いのが気になった。\n特に前半の方は「～と思います。」も連続し過ぎていて、気になるし時間も嵩むため、省いてみてもよいとう。また聞き取りにくい箇所が複数あったため、明瞭な発声を心がけるとよりよくなると思う。
-補足としてより深い部分のお話をしてくださったので、より教師なし学習に興味を持つことができた点。
-1人目の方に関しては、ノイズが入っているのが気になって、集中して聞きづらかった。
-前半の方が少し脱線しがちでわかりにくい部分があったように思う。
-Notebookのやり方に対して自信の感想を述べてくださったり別解を示してくださったので置いてけぼりになることなく履修できました。（教師なし学習）
-一人目の講師の方が、言い間違いや、ボソボソとした話し方をしていて、聞き取りづらく、字幕でもうまく文字起こしが行われなかったので、自分の頭で論理を組み立てながら聞くのが難しかった。その点、二人目の方は、これまでの方と同じようにハキハキと論理立てで説明してくださりわかりやすかった。
-pptスライドとJupyterノートブックのどちらかに偏らず、詳しく説明がなされている点が良かった。
-教師なし学習の方の講師は通信環境が悪いのか、Zoomの問題か、音声にノイズが入ったりして集中力に悪影響が出ていたと感じる。
-講義を２分割して講師を２人にわけると、わかりやすい部分のあるが、時間がどうしても足りなくなる気がした。
-前半の方の講義が難解で、理系の専門用語が多く理解しきれなかった部分がありました。\n全体的に社会人講師の方が初学者に分かりやすくかみ砕いてくださる印象があり、ありがたいです。
-前半パート(教師なし学習)の説明が分かりにくかった。notebookに入ったときに説明が少なすぎて付いていけな

In [14]:
# 講義資料
from google import genai
from google.genai import types
import json

def generate_fb_content(content):
    client = genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location=LOCATION,
    )

    prompt = types.Part.from_text(text=f"""
下記はデータサイエンス講義に関しての受講者からのわかりにくかった部分や改善点に関しての自由記述アンケート結果です。
全ての内容を読み込み、ポジティブ記述とネガティブ記述に分け、ポジティブを更に伸ばしていく示唆と、改善点を提案してください。

** 手順 **
- 自由記述のアンケート結果を読み込む
- 自由記述の内容の講義内容に関係する結果を具体的に分類して集計(大分類-件数-小分類を表示)
- 講義内容に関係する集計結果のみを出力(運営面に関しての意見は出力しない)
- 具体的な講義内容に関して触れている自由記述のコメントを5-10点ほど抜粋
- 講義資料の間違いを指摘するコメントがあれば抜粋する
- 集計結果をもとに示唆出しを行い、講義の内容面での改善施策を提案(講義内容・講義資料以外についての提案は不要)

** 自由記述アンケート結果 **
{content}
全ての内容を読み込み、ポジティブ記述とネガティブ記述に分け、ポジティブを更に伸ばしていく示唆と、改善点を提案してください。

** 手順 **
- 自由記述のアンケート結果を読み込む
- 自由記述の内容の講義内容に関係する結果を具体的に分類して集計(大分類-件数-小分類を表示)
- 講義内容に関係する集計結果のみを出力(運営面に関しての意見は出力しない)
- 具体的な講義内容に関して触れている自由記述のコメントを5-10点ほど抜粋
- 講義資料の間違いを指摘するコメントがあれば抜粋する
- 集計結果をもとに示唆出しを行い、講義の内容面での改善施策を提案(講義内容・講義資料以外についての提案は不要)"""
    )

    model = "gemini-2.5-pro-preview-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[prompt]
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=65535,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
        ]
    )

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )

    return response

In [15]:
# 講師
from google import genai
from google.genai import types
import json

def generate_fb_lecture(content):
    client = genai.Client(
        vertexai=True,
        project=PROJECT_ID,
        location=LOCATION,
    )

    prompt = types.Part.from_text(text=f"""あなたは優秀なアナリスト・コンサルタントです。
下記はデータサイエンス講義に関しての受講者からの講師についての自由記述アンケート結果です。
全ての内容を読み込み、改善点を提案してください。

** 手順 **
- 自由記述のアンケート結果を読み込む
- 自由記述の内容を良かった点と改善点に分割して、それぞれ具体的に分類して集計(大分類-件数-小分類を表示)
- 内容が具体的な自由記述のコメントを5-10点ほど抜粋
- 集計結果をもとに示唆出しを行い、講師に対してのフィードバック、内容の改善施策を提案してください

** 自由記述アンケート結果 **
{content}

上記の内容を読み込み、改善点を提案してください。

** 手順 **
- 自由記述のアンケート結果を読み込む
- 自由記述の内容を良かった点と改善点に分割して、それぞれ具体的に分類して集計(大分類-件数-小分類を表示)
- 内容が具体的な自由記述のコメントを5-10点ほど抜粋
- 集計結果をもとに示唆出しを行い、講師に対してのフィードバック、内容の改善施策を提案してください"""
    )

    model = "gemini-2.5-pro-preview-03-25"
    contents = [
        types.Content(
            role="user",
            parts=[prompt]
        )
    ]

    generate_content_config = types.GenerateContentConfig(
        temperature=1,
        top_p=0.95,
        max_output_tokens=65535,
        response_modalities=["TEXT"],
        safety_settings=[
            types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
            types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF")
        ]
    )

    response = client.models.generate_content(
        model=model,
        contents=contents,
        config=generate_content_config,
    )

    return response

In [19]:
gemini_feedback_content = generate_fb_content(feedback_content)
gemini_feedback_lecturer = generate_fb_lecture(feedback_lecturer)

In [17]:
print(gemini_feedback_content.text)

ご提供いただいたデータサイエンス講義のアンケート結果を分析し、以下の通りポジティブな点の伸長と改善点の提案をまとめました。

### 1. 講義内容に関するフィードバックの分類・集計結果

#### ポジティブ記述 (受講者が価値を感じた点)
- **具体例・応用例 (25件)**
  - 実務・応用のイメージが湧いた (14)
  - 身近な例（金融、医療、CO2等）が良かった (7)
  - 実際のデータを使えたのが良かった (4)
- **講師の説明・講義構成 (24件)**
  - 説明が分かりやすかった・丁寧だった (12)
  - 講義の目標設定や難易度の配慮が良かった (6)
  - 理論と実践（コード）のバランスが良かった (3)
  - 実体験に基づいた話が理解を助けた (2)
  - 復習箇所の明示が良かった (1)
- **講義資料・可視化 (20件)**
  - 図やグラフ、可視化が分かりやすかった (9)
  - スライドが見やすい・整理されていた (7)
  - Notebookの構成が分かりやすかった (4)
- **教師なし学習の理解 (17件)**
  - 概要・全体像が掴めた (10)
  - 教師あり/なしの違いが分かった (4)
  - 面白かった・興味深かった (3)
- **時系列データ分析の理解 (14件)**
  - 興味深かった・学びたい内容だった (6)
  - 概要・全体像が掴めた (6)
  - トレンド・季節・残差への分解が興味深かった/定常性の重要性が理解できた (5)
- **クラスタリング手法の理解 (9件)**
  - クラスタ数決定方法（エルボー法等）が学べて良かった (5)
  - 理論・概要が理解できた (4)
- **ARIMAモデルの理解 (4件)**
  - 構造・理論が理解できた (2)
  - パラメータ選択プロセスが興味深かった (1)
  - 未来予測が感動した (1)
- **主成分分析(PCA)の理解 (3件)**
  - 曖昧だった部分が学べて有意義だった (2)
  - 新しい手法として学べた (1)

#### ネガティブ記述 (受講者が分かりにくいと感じた点)
- **講義全体の難易度・ボリューム (49件)**
  - 内容が難しい・高度すぎる (25)
  - ボリ

In [20]:
print(gemini_feedback_lecturer.text)

承知いたしました。優秀なアナリスト・コンサルタントとして、データサイエンス講義のアンケート結果を分析し、改善提案をいたします。

---

### **データサイエンス講義 講師に関するアンケート分析と改善提案**

#### **1. アンケート内容の分類と集計**

受講者からの自由記述コメントを「良かった点」と「改善点」に分類し、さらに内容ごとに集計しました。

##### **【良かった点】 (計41件)**

*   **説明・解説の質 (24件)**
    *   丁寧・分かりやすい説明: 8
    *   具体例・実務例が豊富: 5
    *   発展的な内容・補足情報: 4
    *   比較による解説（例：教師あり/なし）: 2
    *   重要なポイントの明確化: 2
    *   多様な解法の提示: 2
    *   理論・概念の視覚的説明: 1
*   **講義資料 (8件)**
    *   スライド・Notebookが分かりやすい: 5
    *   図表・グラフが豊富で理解しやすい: 3
*   **講師の姿勢・話し方 (7件)**
    *   声が明瞭で聞き取りやすい: 3
    *   熱意・親身な姿勢: 3
    *   誠実さ・信頼感: 1
*   **講義構成 (2件)**
    *   講師が複数でメリハリがあった: 2

##### **【改善点】 (計66件)**

*   **説明の分かりにくさ (28件)**
    *   初学者への配慮不足・専門用語の多用: 9
    *   コードの説明不足・省略: 6
    *   ポイントが不明確・何を言いたいか分かりにくい: 5
    *   説明が駆け足・省略が多い: 4
    *   話がまとまっていない・脱線: 2
    *   単調・読み上げに終始: 2
*   **話し方・音声環境 (25件)**
    *   音声品質（ノイズ、音割れ、こもる）: 12
    *   声が聞き取りにくい（ボソボソ、小さい）: 6
    *   フィラー（「えっと」「まあ」）や口癖: 4
    *   話すスピードが速い: 2
    *   指示語が分かりにくい: 1
*   **講義構成・時間管理 (10件)**
    *  

### driveにドキュメントとして保存

In [ ]:
# # 必要なライブラリのインストール
# !pip install --upgrade google-api-python-client

# from googleapiclient.discovery import build
# from google.colab import auth
# from google.auth import default
# import sys

# def find_folder_id_by_path(drive_service, folder_path):
#     """指定されたパスのフォルダIDを取得する関数"""
#     if not folder_path or folder_path == '/':
#         # ルートフォルダの場合
#         return 'root'

#     # パスを各フォルダ名に分割
#     folders = folder_path.strip('/').split('/')

#     parent_id = 'root'
#     for folder in folders:
#         query = f"name='{folder}' and mimeType='application/vnd.google-apps.folder' and '{parent_id}' in parents and trashed=false"
#         results = drive_service.files().list(q=query, spaces='drive', fields="files(id, name)").execute()
#         items = results.get('files', [])

#         if not items:
#             # フォルダが存在しない場合は作成
#             folder_metadata = {
#                 'name': folder,
#                 'mimeType': 'application/vnd.google-apps.folder',
#                 'parents': [parent_id]
#             }
#             new_folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
#             parent_id = new_folder.get('id')
#         else:
#             # 既存のフォルダを使用
#             parent_id = items[0]['id']

#     return parent_id

# def create_document(title, content, folder_path):
#     # Google Colabでの認証
#     auth.authenticate_user()
#     creds, _ = default()

#     # Google Docs APIとDrive APIのサービスを初期化
#     docs_service = build('docs', 'v1', credentials=creds)
#     drive_service = build('drive', 'v3', credentials=creds)

#     # 空のドキュメントを作成
#     document = docs_service.documents().create(
#         body={'title': title}
#     ).execute()
#     doc_id = document['documentId']

#     # ドキュメントにテキストを挿入
#     requests = [
#         {
#             'insertText': {
#                 'location': {
#                     'index': 1
#                 },
#                 'text': content
#             }
#         }
#     ]

#     docs_service.documents().batchUpdate(
#         documentId=doc_id,
#         body={'requests': requests}
#     ).execute()

#     # 指定したフォルダに移動
#     folder_id = find_folder_id_by_path(drive_service, folder_path)

#     # ドキュメントの親フォルダを更新
#     file = drive_service.files().get(fileId=doc_id, fields='parents').execute()
#     previous_parents = ",".join(file.get('parents'))

#     # ファイルを指定フォルダに移動
#     drive_service.files().update(
#         fileId=doc_id,
#         addParents=folder_id,
#         removeParents=previous_parents,
#         fields='id, parents'
#     ).execute()

#     print(f"ドキュメント '{title}' がフォルダパス '{folder_path}' に作成されました")
#     print(f"ドキュメントID: {doc_id}")
#     print(f"ドキュメントURL: https://docs.google.com/document/d/{doc_id}/edit")

#     return doc_id

# # 使用例
# document_title = f"第{section_num}回_講師フィードバック"
# document_content = gemini_feedback_lecturer.text
# folder_path = "Beginning LLM Level.6"  # 指定されたパス

# create_document(document_title, document_content, folder_path)

ドキュメント '第0回_講師フィードバック' がフォルダパス 'Beginning LLM Level.6' に作成されました
ドキュメントID: 143NDK-OqbKlqqiAK8ShxxCWcmckfHEF9VZkUKoT_T_U
ドキュメントURL: https://docs.google.com/document/d/143NDK-OqbKlqqiAK8ShxxCWcmckfHEF9VZkUKoT_T_U/edit


'143NDK-OqbKlqqiAK8ShxxCWcmckfHEF9VZkUKoT_T_U'